# Import

These requirements are necessary if you launch this notebook from SageMaker instances

In [1]:
"""!pip install mlflow
!pip install pytorch-lightning
!pip install transformers
!pip install tqdm
!pip install sagemaker
!pip install s3fs
!pip install smdebug"""

'!pip install mlflow\n!pip install pytorch-lightning\n!pip install transformers\n!pip install tqdm\n!pip install sagemaker\n!pip install s3fs\n!pip install smdebug'

In [2]:
import sys
sys.path.append('../../../')

import os
import sys
import logging
import argparse
from pathlib import Path
from ast import literal_eval
from collections import Counter
from typing import Any, Dict, Optional

In [3]:
from tqdm.auto import tqdm

import torchmetrics
from torchmetrics.functional import accuracy, f1, auroc

import sagemaker
from sagemaker import get_execution_role
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.core.decorators import auto_move_data
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import MLFlowLogger


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix


import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc

import transformers
from transformers import (
    AdamW,
    AutoConfig,
    AutoModel,
    AutoTokenizer,
)
from transformers.optimization import (
    Adafactor,
    get_linear_schedule_with_warmup,
)

Local constants, regarding the data, MLFlow server, paths, etc..: use them

In [4]:
from deep.constants import *
from deep.utils import *

/home/selim/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
%load_ext autoreload
%autoreload 2

# Data

You can use the data you want. We advise the `pandas` format.

In [6]:
TRAIN_PATH = os.path.join('..', '..', '..', "data", "frameworks_data", "data_v0.4.4", "data_v0.4.4_train.csv")
VAL_PATH = os.path.join('..', '..', '..', "data", "frameworks_data", "data_v0.4.4", "data_v0.4.4_val.csv")
TEST_PATH = os.path.join('..', '..', '..', "data", "frameworks_data", "data_v0.4.4", "data_v0.4.4_test.csv")

train_df = pd.read_csv(TRAIN_PATH)
val_df = pd.concat([pd.read_csv(TEST_PATH), pd.read_csv(VAL_PATH)])

## Sagemaker Prep

### Session

Configure SageMaker

In [7]:
sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
role = SAGEMAKER_ROLE
role_arn = SAGEMAKER_ROLE_ARN
tracking_uri = MLFLOW_SERVER

### Bucket upload

You need to upload data to an S3 bucket. 




In [8]:
MLFLOW_SERVER

'http://mlflow-deep-387470f3-1883319727.us-east-1.elb.amazonaws.com/'

In [9]:
sample = False  # To make the computations faster, sample = True.

if sample:
    train_df = train_df.sample(n=1000)
    val_df = val_df.sample(n=1000)
    
job_name = f"pytorch-{formatted_time()}-subpillars-model-test-mlflow"  # change it as you prefer
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train.pickle')
val_path = str(input_path / 'val.pickle')


train_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
val_df.to_pickle(val_path, protocol=4)

### Estimator Definition

In [10]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [11]:
from sagemaker.pytorch import PyTorch


hyperparameters={
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': "pl-sectors",
    'max_len': 256,
    'epochs': 5,
    'model_name': 'microsoft/xtremedistil-l6-h384-uncased',
    'tokenizer_name': 'microsoft/xtremedistil-l6-h384-uncased',
    'dropout_rate': 0.3,
    'pred_threshold':0.4,
    'output_length': 384,
    'learning_rate': 7e-5,
    'training_column':'sectors'
}

estimator = PyTorch(
    entry_point='train_mlflow.py',
    source_dir=str('../../../scripts/training/selim/multiclass-lightning'),
    output_path=str(DEV_BUCKET/'models/'),
    code_location=str(input_path),
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=role,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    job_name=job_name,
#     train_instance_count=2,
#     train_instance_type="ml.c4.xlarge",
)

In [12]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [13]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

2021-08-09 09:50:14 Starting - Starting the training job...
2021-08-09 09:50:37 Starting - Launching requested ML instancesProfilerReport-1628502611: InProgress
......
2021-08-09 09:51:37 Starting - Preparing the instances for training......
2021-08-09 09:52:57 Downloading - Downloading input data...
2021-08-09 09:53:17 Training - Downloading the training image...................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-08-09 09:57:02,762 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-08-09 09:57:02,786 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-08-09 09:57:09,022 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-08-09 09:57:09,576 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt

2021-08-0

  Created wheel for nltk: filename=nltk-3.2.5-py3-none-any.whl size=1392142 sha256=169d55f74fedee10b340189885c84e98d1d996daae71cdb74376f695fa0cdcbf
  Stored in directory: /root/.cache/pip/wheels/f2/7f/71/cb36468789a03b5e2908281c8e1ce093e6860258b6b61677d8
  Created wheel for sagemaker: filename=sagemaker-2.49.1-py2.py3-none-any.whl size=591917 sha256=43bc9b3af4e3255881314f381455c3fdd2edfadd84d533ad55c4fb16a4783f94
  Stored in directory: /root/.cache/pip/wheels/4c/af/ea/8ff5943a87155df5b184e54474fbf2b59b75e5c172854643c6
  Created wheel for aiobotocore: filename=aiobotocore-1.3.3-py3-none-any.whl size=48336 sha256=659977701541d018dc6e124c98b623dce2a5b5814e05ddf10a6d2a6c21162715
  Stored in directory: /root/.cache/pip/wheels/b8/12/24/4cc15fc7666d3811f9627ea675972ff0c6d3d56bd5b53aa8ab
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=c5934eb2393142748eaa4580353be0f4c832c3493286bd0f3393be4514c860d1
  Stored in directory: /root/.cache/pip/wheels/e9/7b

  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3161 sha256=ff359e6bd881176f5fcf7f17c11fd3692aea6e3fb6ba24a1922725be896670a7
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=f6220907c1505fd7e22be4ef7cb0ed0184c8148d54ea5b346ef50f910eb93c27
  Stored in directory: /root/.cache/pip/wheels/93/2a/eb/e58dbcbc963549ee4f065ff80a59f274cc7210b6eab962acdc
  Created wheel for wrapt: filename=wrapt-1.12.1-cp36-cp36m-linux_x86_64.whl size=69744 sha256=2e7fb0e1d6280e20bcd4ef6969deee0fe8a1772729dddc209694c20f498d1646
  Stored in directory: /root/.cache/pip/wheels/32/42/7f/23cae9ff6ef66798d00dc5d659088e57dbba01566f6c60db63
  Created wheel for prometheus-flask-exporter: filename=prometheus_flask_exporter-0.18.2-py3-none-any.whl size=17398 sha256=3146187f5d7357b5b26f427678df6f11b44faaed1b536d1fcb7129d0b3d85692
  Stored in directory:

#015Validation sanity check: 0it [00:00, ?it/s]#015Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s][2021-08-09 10:01:40.964 algo-1:91 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-08-09 10:01:41.004 algo-1:91 INFO profiler_config_parser.py:102] User has disabled profiler.
[2021-08-09 10:01:41.005 algo-1:91 INFO json_config.py:91] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
[2021-08-09 10:01:41.006 algo-1:91 INFO hook.py:201] tensorboard_dir has not been set for the hook. SMDebug will not be exporting tensorboard summaries.
[2021-08-09 10:01:41.006 algo-1:91 INFO hook.py:255] Saving to /opt/ml/output/tensors
[2021-08-09 10:01:41.006 algo-1:91 INFO state_store.py:77] The checkpoint config file /opt/ml/input/config/checkpointconfig.json does not exist.
[2021-08-09 10:01:41.184 algo-1:91 INFO hook.py:594] name:model.l0.embeddings.word_embeddings.weight count_params:11720448
[2021-08-09 10:01:41.184 algo-1:91 INFO hook.py:594] nam

#015                                                              #015#015Training: 0it [00:00, ?it/s]#015Training:   0%|          | 0/3737 [00:00<?, ?it/s]#015Epoch 0:   0%|          | 0/3737 [00:00<?, ?it/s] #015Epoch 0:   1%|          | 30/3737 [00:02<05:25, 11.40it/s]#015Epoch 0:   1%|          | 30/3737 [00:02<05:25, 11.39it/s, loss=1.12, v_num=0, val_f1_epoch=0.108, val_loss_epoch=0.686, train_f1=0.552]#015Epoch 0:   2%|▏         | 60/3737 [00:04<04:58, 12.30it/s, loss=1.12, v_num=0, val_f1_epoch=0.108, val_loss_epoch=0.686, train_f1=0.552]#015Epoch 0:   2%|▏         | 60/3737 [00:04<04:58, 12.30it/s, loss=0.713, v_num=0, val_f1_epoch=0.108, val_loss_epoch=0.686, train_f1=0.477]#015Epoch 0:   2%|▏         | 90/3737 [00:07<04:45, 12.79it/s, loss=0.713, v_num=0, val_f1_epoch=0.108, val_loss_epoch=0.686, train_f1=0.477]#015Epoch 0:   2%|▏         | 90/3737 [00:07<04:45, 12.79it/s, loss=0.692, v_num=0, val_f1_epoch=0.108, val_loss_epoch=0.686, train_f1=0.524]#015Epoch 0:   3%|▎      

...

#015Validating: 100%|██████████| 340/340 [00:12<00:00, 28.30it/s]#033[A#015Epoch 3: 100%|██████████| 3737/3737 [04:23<00:00, 14.21it/s, loss=0.244, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.865, val_f1_step=0.940, val_loss_step=0.0774]
#015                                                             #033[A#015Epoch 3:   0%|          | 0/3737 [00:00<?, ?it/s, loss=0.244, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.865, val_f1_step=0.940, val_loss_step=0.0774]           #015Epoch 4:   0%|          | 0/3737 [00:00<?, ?it/s, loss=0.244, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.865, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:   1%|          | 30/3737 [00:02<05:07, 12.05it/s, loss=0.244, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.865, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:   1%|          | 30/3737 [00:02<05:07, 12.04it/s, loss=0.213, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1

 val_loss_epoch=0.102, train_f1=0.893, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  35%|███▌      | 1320/3737 [01:37<02:58, 13.58it/s, loss=0.219, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.923, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  36%|███▌      | 1350/3737 [01:39<02:55, 13.59it/s, loss=0.219, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.923, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  36%|███▌      | 1350/3737 [01:39<02:55, 13.59it/s, loss=0.196, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.942, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  37%|███▋      | 1380/3737 [01:41<02:53, 13.59it/s, loss=0.196, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.942, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  37%|███▋      | 1380/3737 [01:41<02:53, 13.59it/s, loss=0.212, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.957, val_f1_step=0.940, val_loss_step=0.0774]#

 val_loss_step=0.0774]  #015Epoch 4:  70%|██████▉   | 2610/3737 [03:11<01:22, 13.60it/s, loss=0.2, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.865, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  70%|██████▉   | 2610/3737 [03:11<01:22, 13.60it/s, loss=0.211, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.880, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  71%|███████   | 2640/3737 [03:14<01:20, 13.60it/s, loss=0.211, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.880, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  71%|███████   | 2640/3737 [03:14<01:20, 13.60it/s, loss=0.229, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.949, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  71%|███████▏  | 2670/3737 [03:16<01:18, 13.59it/s, loss=0.229, v_num=0, val_f1_epoch=0.915, val_loss_epoch=0.102, train_f1=0.949, val_f1_step=0.940, val_loss_step=0.0774]#015Epoch 4:  71%|███████▏  | 2670/3737 [03:16<01:18, 13.5

#015Validating: 100%|██████████| 340/340 [00:12<00:00, 28.09it/s]#033[A#015Epoch 4: 100%|██████████| 3737/3737 [04:22<00:00, 14.24it/s, loss=0.215, v_num=0, val_f1_epoch=0.919, val_loss_epoch=0.0979, train_f1=0.903, val_f1_step=0.945, val_loss_step=0.0708]
#015                                                             #033[A#015Epoch 4: 100%|██████████| 3737/3737 [04:23<00:00, 14.16it/s, loss=0.215, v_num=0, val_f1_epoch=0.919, val_loss_epoch=0.0979, train_f1=0.903, val_f1_step=0.945, val_loss_step=0.0708]

2021-08-09 10:24:45 Uploading - Uploading generated training model(21649, 11)
(21649, 11)
2021-08-09 09:58:27.099723: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
INFO:root:reading, preprocessing data
2021/08/09 09:58:31 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of pytorch. If you encounter errors during autologging, try upgrading / downgrading pytorch to a supported version


2021-08-09 10:25:22 Completed - Training job completed
Training seconds: 1957
Billable seconds: 1957
